#### install packages / import Libraries:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from keras import backend as K
import tensorflow as tf
import cv2
import pandas as pd
import math
from patchify import patchify,unpatchify
import tifffile as tif
from tqdm import tqdm
import time
import pyvips
from datetime import datetime
import datetime as dt
import pyvips
import utils

c:\Users\Administrator\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Functions

In [3]:
def load_external_test_slides_to_df(root = 'test_images/'):
    file_dir = sorted(os.listdir(root))
    dir_path = [root + x for x in file_dir if x!='desktop.ini']
    ext_test_imgs_df = pd.DataFrame()
    img_path_list = []
    gt_path_list = []
    img_dim_list = []
    gt_dim_list = []
    img_dim_list = []
    gt_dim_list = []
    for dr in dir_path:
        contents = sorted(os.listdir(dr))
        img_path_list.append(dr+'/'+contents[0])
        gt_path_list.append(dr+'/'+contents[1])

    for img_path, gt_path in zip(img_path_list,gt_path_list):
        img = pyvips.Image.tiffload(img_path)
        gt = pyvips.Image.tiffload(gt_path)
        img_dim = (img.height,img.width)
        gt_dim = (gt.height,gt.width)
        img_dim_list.append(img_dim)
        gt_dim_list.append(gt_dim)
        del img
        del gt


        
    ext_test_imgs_df['img_file'] = [fname.split('/')[-1] for fname in img_path_list]
    ext_test_imgs_df['gt_file'] = [fname.split('/')[-1] for fname in gt_path_list]
    ext_test_imgs_df['img_path'] = img_path_list
    ext_test_imgs_df['gt_path'] = gt_path_list

    ext_test_imgs_df['img_dims'] = img_dim_list
    ext_test_imgs_df['gt_dims'] = gt_dim_list


    return ext_test_imgs_df

In [4]:
def load_dataset_test_slides_to_df(img_root = 'test_tiff/'):
    gt_root = 'test_dataset_mask/'
    img_files_dir = sorted(os.listdir(img_root))
    gt_files_dir = sorted(os.listdir(gt_root))
    img_paths_list = [img_root + x for x in img_files_dir if x!='desktop.ini']
    gt_paths_list = [gt_root + x for x in gt_files_dir if x!='desktop.ini']
    img_dim_list = []
    gt_dim_list = []
    dataset_test_imgs_df = pd.DataFrame()
    for img_path, gt_path in zip(img_paths_list,gt_paths_list):
        img = pyvips.Image.tiffload(img_path)
        gt = pyvips.Image.tiffload(gt_path)
        img_dim = (img.height,img.width)
        gt_dim = (gt.height,gt.width)
        img_dim_list.append(img_dim)
        gt_dim_list.append(gt_dim)

    dataset_test_imgs_df['img_file'] = [fname.split('/')[-1] for fname in img_paths_list]
    dataset_test_imgs_df['img_path'] = img_paths_list
    dataset_test_imgs_df['img_dims'] = img_dim_list
    dataset_test_imgs_df['gt_file'] = [fname.split('/')[-1] for fname in gt_paths_list]
    dataset_test_imgs_df['gt_path'] = gt_paths_list
    dataset_test_imgs_df['gt_dims'] = gt_dim_list
    return dataset_test_imgs_df

In [5]:
def load_models(root = 'models/'):
    unet_all_model = tf.keras.models.load_model(root+'unet_all.h5', custom_objects={'mean_iou':utils.mean_iou})
    unet_filtered_98_3_model = tf.keras.models.load_model(root+'unet_filtered_98-3.h5', custom_objects={'mean_iou':utils.mean_iou})
    unet_filtered_90_20_model = tf.keras.models.load_model(root+'unet_filtered_90-20.h5', custom_objects={'mean_iou':utils.mean_iou})
    unet_filtered_100_50_model = tf.keras.models.load_model(root+'unet_filtered_100-50.h5', custom_objects={'mean_iou':utils.mean_iou})
    VGG_unet_filtered_90_20_model = tf.keras.models.load_model(root+'VGG_unet_filtered_90-20.h5', custom_objects={'mean_iou':utils.mean_iou,'dice_loss':utils.dice_loss})
    res_unet_100_0_model = tf.keras.models.load_model(root+'res_unet_100_0.h5', custom_objects={'mean_iou':utils.mean_iou})

    models = {
        'unet_100_0_model':{'model':unet_all_model, 'name':'Unet','trained_dataset_range':(0,100) },
        'unet_98_3_model':{'model':unet_filtered_98_3_model, 'name':'Unet','trained_dataset_range':(3,98) },
        'unet_90_20_model':{'model':unet_filtered_90_20_model, 'name':'Unet','trained_dataset_range':(20,90) },
        'unet_100_50_model':{'model':unet_filtered_100_50_model, 'name':'Unet','trained_dataset_range':(50,100) },
        'VGG_unet_90_20_model':{'model':VGG_unet_filtered_90_20_model, 'name':'VGG_Unet','trained_dataset_range':(20,90) },
        'res_unet_100_0_model':{'model':res_unet_100_0_model, 'name':'Res_Unet','trained_dataset_range':(0,100) },
    }
    return models

In [6]:
slides_df = load_external_test_slides_to_df()
slides_df

,img_file,gt_file,img_path,gt_path,img_dims,gt_dims
0,102.tif,102_mask.tiff,test_images/102/102.tif,test_images/102/102_mask.tiff,"(6397, 26002)","(6397, 26002)"
1,121.tif,121_mask.tiff,test_images/121/121.tif,test_images/121/121_mask.tiff,"(17176, 6573)","(17176, 6573)"
2,201.tif,201_mask.tiff,test_images/201/201.tif,test_images/201/201_mask.tiff,"(5461, 23871)","(5461, 23871)"
3,202.tif,202_mask.tiff,test_images/202/202.tif,test_images/202/202_mask.tiff,"(4371, 6179)","(4371, 6179)"
4,203.tif,203_mask.tiff,test_images/203/203.tif,test_images/203/203_mask.tiff,"(5591, 3546)","(5591, 3546)"
5,204.tif,204_mask.tiff,test_images/204/204.tif,test_images/204/204_mask.tiff,"(5253, 12053)","(5253, 12053)"
6,355.tiff,355_mask.tiff,test_images/355/355.tiff,test_images/355/355_mask.tiff,"(5412, 7215)","(5412, 7215)"
7,356.tiff,356_mask.tiff,test_images/356/356.tiff,test_images/356/356_mask.tiff,"(5412, 7215)","(5412, 7215)"
8,358.tiff,358_mask.tiff,test_images/358/358.tiff,test_images/358/358_mask.tiff,"(5412, 7215)","(5412, 7215)"
9,359.tiff,359_mask.tiff,test_images/359/359.tiff,test_images/359/359_mask.tiff,"(5412, 7215)","(5412, 7215)"


In [7]:
# create dataFrame to save the testing results

columns = ['image', 'dimensions', 'model','trained_dataset_range','pixel_accuracy_th', 'iou','test_date','duration']
df = pd.DataFrame([], columns=columns)
df

,image,dimensions,model,trained_dataset_range,pixel_accuracy_th,iou,test_date,duration


In [8]:
# load models and save the name of model in list
# except the VGG and ResNet

models = load_models()
models_list = list(models.keys())[:-2]
models_list

['unet_100_0_model',
 'unet_98_3_model',
 'unet_90_20_model',
 'unet_100_50_model']

### Get the predicted images

In [12]:
for i in range(len(models_list)):
    print(f'Prediction for {models_list[i]}........')
    for idx, row in slides_df.iterrows():
        result, pred_mask, df = utils.whole_img_iou(row,models[models_list[i]], 0.5, df)

        if not os.path.exists('predicted_images'):
            os.mkdir('predicted_images')
        if not os.path.exists(f'predicted_images/{models_list[i]}'):
            os.mkdir(f'predicted_images/{models_list[i]}')
        cv2.imwrite(f'predicted_images/{models_list[i]}/{row["img_file"][:3]}.png', pred_mask)
        
        if not os.path.exists(f'whole_slide_mask_before/{models_list[i]}'):
            os.mkdir(f'whole_slide_mask_before/{models_list[i]}')
        cv2.imwrite(f'whole_slide_mask_before/{models_list[i]}/{row["img_file"][:3]}.png', result)
    

Prediction for unet_100_0_model........


100%|██████████| 12/12 [01:58<00:00,  9.85s/it]


IOU : 0.6708603803262431
['102.tif', (6397, 26002), 'Unet', (0, 100), 0.5, 0.6709, '21/06/2023 18:19:07', '0:01:58.536152']


100%|██████████| 33/33 [01:23<00:00,  2.52s/it]


IOU : 0.7557147080049706
['121.tif', (17176, 6573), 'Unet', (0, 100), 0.5, 0.7557, '21/06/2023 18:21:08', '0:01:23.284892']


...

IOU : 0.476681574714066
['401.tif', (1127, 805), 'Unet', (50, 100), 0.5, 0.4767, '21/06/2023 19:54:04', '0:00:00.931146']


100%|██████████| 7/7 [00:21<00:00,  3.11s/it]


IOU : 0.008324813639485897
['402.tif', (3656, 4021), 'Unet', (50, 100), 0.5, 0.0083, '21/06/2023 19:54:05', '0:00:21.836803']


100%|██████████| 1/1 [00:00<00:00,  1.98it/s]

IOU : 0.6022117095876242
['403.tif', (746, 772), 'Unet', (50, 100), 0.5, 0.6022, '21/06/2023 19:54:27', '0:00:00.514354']


In [14]:

df.to_csv('slide_results.csv', mode='a', index=False, header=False)
columns = ['image', 'dimensions', 'model','trained_dataset_range','pixel_accuracy_th', 'iou','test_date','duration']
df = pd.DataFrame([], columns=columns)
df

,image,dimensions,model,trained_dataset_range,pixel_accuracy_th,iou,test_date,duration


In [14]:
columns = ['image', 'dimensions', 'model','trained_dataset_range','pixel_accuracy_th', 'iou','test_date','duration']
df = pd.read_csv('slide_results.csv', names=columns)
# df.to_csv('slide_results.csv', mode='a', index=False, header=False)

,image,dimensions,model,trained_dataset_range,pixel_accuracy_th,iou,test_date,duration
0,102.tif,"(6397, 26002)",Unet,"(0, 100)",0.5,0.5920,21/06/2023 14:17:42,0:00:40.280827
1,121.tif,"(17176, 6573)",Unet,"(0, 100)",0.5,0.7197,21/06/2023 14:18:25,0:00:31.132834
2,201.tif,"(5461, 23871)",Unet,"(0, 100)",0.5,0.7363,21/06/2023 14:18:59,0:00:46.934618
3,202.tif,"(4371, 6179)",Unet,"(0, 100)",0.5,0.7829,21/06/2023 14:19:48,0:00:06.971996
4,203.tif,"(5591, 3546)",Unet,"(0, 100)",0.5,0.8453,21/06/2023 14:19:55,0:00:04.801503
...,...,...,...,...,...,...,...,...
95,395.tiff,"(5412, 7215)",Unet,"(50, 100)",0.5,0.0452,21/06/2023 14:39:36,0:00:10.199326
96,399.tiff,"(5412, 7215)",Unet,"(50, 100)",0.5,0.0640,21/06/2023 14:39:48,0:00:12.774646
97,401.tif,"(1127, 805)",Unet,"(50, 100)",0.5,0.5544,21/06/2023 14:40:01,0:00:00.227145
98,402.tif,"(3656, 4021)",Unet,"(50, 100)",0.5,0.0021,21/06/2023 14:40:02,0:00:03.535211
